In [73]:
import pandas as pd
import utils
import datetime as dt
from dateutil.parser import *

In [74]:
df_trades = pd.read_pickle("USD_JPY_H4_trades.pkl")

In [75]:
pair = "USD_JPY"

In [76]:
df_raw = pd.read_pickle(utils.get_his_data_filename(pair, "M5"))

In [77]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df_raw.columns if x not in non_cols]
df_raw[mod_cols] = df_raw[mod_cols].apply(pd.to_numeric)

In [78]:
df_trades["time"] = [parse(x) for x in df_trades.time]
df_raw["time"] = [parse(x) for x in df_raw.time]

In [79]:
df_trades["next"] = df_trades["time"].shift(-1)

In [80]:
df_trades["trade_end"] = df_trades.next + dt.timedelta(hours=3, minutes=55)
df_trades['trade_start'] = df_trades.time + dt.timedelta(hours=4)

In [81]:
df_trades[['time', 'next', 'trade_end', 'trade_start']].head()

,time,next,trade_end,trade_start
0,2020-01-02 02:00:00+00:00,2020-01-02 18:00:00+00:00,2020-01-02 21:55:00+00:00,2020-01-02 06:00:00+00:00
4,2020-01-02 18:00:00+00:00,2020-01-06 02:00:00+00:00,2020-01-06 05:55:00+00:00,2020-01-02 22:00:00+00:00
12,2020-01-06 02:00:00+00:00,2020-01-06 10:00:00+00:00,2020-01-06 13:55:00+00:00,2020-01-06 06:00:00+00:00
14,2020-01-06 10:00:00+00:00,2020-01-07 10:00:00+00:00,2020-01-07 13:55:00+00:00,2020-01-06 14:00:00+00:00
20,2020-01-07 10:00:00+00:00,2020-01-07 18:00:00+00:00,2020-01-07 21:55:00+00:00,2020-01-07 14:00:00+00:00


In [82]:
df_trades.dropna(inplace=True)
df_trades.reset_index(drop=True, inplace=True)

In [86]:
def signal_text(signal):
    if signal == 1:
        return 'BUY'
    elif signal == -1:
        return 'SELL'
    return 'NONE'

def triggered(direction, current_price, signal_price):
    if direction == 1 and current_price > signal_price:
        return True
    elif direction == -1 and current_price < signal_price:
        return True
    return False

def process_buy(start_index, TP, SL, prices, start_price):
    for price in prices[start_index:]:
        if price <= TP:
            return 2.0
        elif price >= SL:
            return -1.0
    return 0.0

def process_sell(start_index, TP, SL, prices, start_price):
    for price in prices[start_index:]:
        if price >= TP:
            return 2.0
        elif price <= SL:
            return -1.0
    return 0.0

def process_trade(start_index, direction, TP, SL, prices, start_price):
    if direction == 1:
        return process_buy(start_index, TP, SL, prices, start_price)
    else:
        return process_sell(start_index, TP, SL, prices, start_price)

def process_m5(m5_df, row):
    result = 0.0
    for index, price in enumerate(m5_df.mid_c.values):
        if triggered(row.SIGNAL, price, row.ENTRY) == True:
            print(f"Signal at {price:.2f} {row.ENTRY:.2f} {row.SIGNAL:.2f}")
            result = process_trade(index, row.SIGNAL, row.TAKEPROFIT, row.STOPLOSS, m5_df.mid_c.values, row.ENTRY)
            break
    return result

In [87]:
total = 0
for index, row in df_trades.iterrows():
    m5_data = df_raw[(df_raw.time >= row.trade_start) & (df_raw.time <= row.trade_end)]
    print(f"{row.time} {signal_text(row.SIGNAL)} {row.ENTRY:.2f} {m5_data.shape}")
    total += process_m5(m5_data, row)
    if index > 10:
        break
print(total)

2020-01-02 02:00:00+00:00 BUY 108.78 (192, 14)
Signal at 108.79 108.78 1.00
2020-01-02 18:00:00+00:00 SELL 108.16 (384, 14)
Signal at 108.13 108.16 -1.00
2020-01-06 02:00:00+00:00 BUY 108.16 (96, 14)
Signal at 108.16 108.16 1.00
2020-01-06 10:00:00+00:00 SELL 107.90 (286, 14)
2020-01-07 10:00:00+00:00 BUY 108.52 (96, 14)
Signal at 108.54 108.52 1.00
2020-01-07 18:00:00+00:00 BUY 108.66 (96, 14)
2020-01-08 02:00:00+00:00 SELL 107.56 (768, 14)
2020-01-10 18:00:00+00:00 SELL 109.41 (374, 14)
2020-01-14 02:00:00+00:00 BUY 110.24 (96, 14)
2020-01-14 10:00:00+00:00 SELL 109.86 (191, 14)
Signal at 109.86 109.86 -1.00
2020-01-15 02:00:00+00:00 SELL 109.80 (192, 14)
2020-01-15 18:00:00+00:00 BUY 110.02 (95, 14)
10.0
